In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

In [ ]:
# Most Stats Available
data_f = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wire_tracks_data_1k.root"
mc_f = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wire_tracks_mc_1k.root"
c_f = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wire_tracks_corsika_shw20m_b2.root"

print("Got MC and Data Files to compare and analyze")

# Load the Track DFs

In [ ]:
inFileMC = uproot.open(mc_f)
mc_trk_tree = inFileMC["track_tree"]
mc_df = mc_trk_tree.arrays(mc_trk_tree.keys(), library="pd")
mc_df[:2]

In [ ]:
inFileC = uproot.open(c_f)
c_trk_tree = inFileC["track_tree"]
c_df = c_trk_tree.arrays(c_trk_tree.keys(), library="pd")
c_df[:2]

In [ ]:
inFileData = uproot.open(data_f)
data_trk_tree = inFileData["track_tree"]
data_df = data_trk_tree.arrays(data_trk_tree.keys(), library="pd")
data_df[:2]

# Helper Functions

In [ ]:
def plot_1D_error(df, var, B, S, label=None, color=None, linestyle='-', linewidth=2, density=True, event_norm=False):
    """
    Plot a 1D histogram with error bands.
    
    Parameters:
    - df: DataFrame containing the data.
    - var: Column name to plot.
    - B: Bin edges for the histogram.
    - S: Selection condition for filtering the DataFrame.
    - label: Label for the histogram.
    - color: Color of the histogram.
    - linestyle: Line style for the histogram.
    - linewidth: Width of the line.
    - density: Whether to normalize the histogram.
    """

    counts, bin_edges, errors = 0, 0, 0
    if density:
        # Get histogram data without plotting
        counts, bin_edges = np.histogram(df.query(S)[var].values, bins=B)
        errors = np.sqrt(counts)/ np.sum(counts)  # Poisson error
        counts = counts / np.sum(counts)
       

    elif event_norm:
        # Get histogram data without plotting, normalized by number of events
        counts, bin_edges = np.histogram(df.query(S)[var].values, bins=B)
        temp_df = df.query(S)
        unique_count = temp_df.drop_duplicates(subset=["run", "subrun", "evt"]).shape[0]
        errors = np.sqrt(counts) / unique_count
        counts = counts / unique_count
    else:
        counts, bin_edges = np.histogram(df.query(S)[var].values, bins=B)
        errors = np.sqrt(counts)


    # Bin centers for plotting
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # Plot the histogram as a step line
    plt.step(bin_centers, counts, where='mid', color=color, label=label)

    # Plot shaded error band
    plt.fill_between(bin_centers, counts - errors, counts + errors,
                 color=color, alpha=0.3, step='mid')

  
print("made 1D plotting function")


# Track Directions

In [ ]:
DATA_LABEL = "Data"
MC_LABEL = "MC"

DATA_COLOR = "blue"
MC_COLOR = "orange"

print("Made Labels and Colors")

In [ ]:
S = "sel > 0"
wx = 4
wy = 4
Bx = np.arange(-200, 200+wx, wx)
By = np.arange(-200, 200+wy, wy)


plt.hist2d(data_df.query(S)['sx'].values, data_df.query(S)['sy'].values, bins=(Bx, By), 
           density=True, norm=LogNorm(), cmap='viridis')

plt.colorbar(label='Normalized')
plt.xlabel('Track Start X [cm]', fontsize=20)
plt.ylabel('Track Start Y [cm]', fontsize=20)
plt.title('Data: Selection >= 1', fontsize=20)
plt.grid()
plt.show() 

In [ ]:
S = "sel > 0"
wx = 4
wy = 4
Bx = np.arange(-200, 200+wx, wx)
By = np.arange(-200, 200+wy, wy)


plt.hist2d(mc_df.query(S)['sx'].values, mc_df.query(S)['sy'].values, bins=(Bx, By), 
           density=True, norm=LogNorm(), cmap='viridis')

plt.colorbar(label='Normalized')
plt.xlabel('Track Start X [cm]', fontsize=20)
plt.ylabel('Track Start Y [cm]', fontsize=20)
plt.title('MC: Selection >= 1', fontsize=20)
plt.grid()
plt.show() 

In [ ]:
S = "sel >= 1 and rr > 60"
bin_w = 10
B = np.arange(-200, 200+bin_w, bin_w)
plot_1D_error(data_df, "sx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "sx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Start X [cm]', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Selection >= 1', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 1 and rr > 60"
bin_w = 10
B = np.arange(-200, 200+bin_w, bin_w)
plot_1D_error(data_df, "sy", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "sy", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Start Y [cm]', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Selection >= 1', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 1 and rr > 60"
bin_w = 10
B = np.arange(0, 500+bin_w, bin_w)
plot_1D_error(data_df, "sz", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "sz", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Start Z [cm]', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Selection >= 1', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Stopping Muon Sample: Selection = 0', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 1"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Anode-Cathode Crossers: Selection = 1', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Other Cathode Crossers: Selection = 2', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR, event_norm=True, density=False)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR, event_norm=True, density=False)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Stopping Muon Sample: Selection = 0', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 1"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR, density=False, event_norm=True)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR, density=False, event_norm=True)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Anode-Cathode Crossers: Selection = 1', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR, density=False, event_norm=True)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR, density=False, event_norm=True)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Other Cathode Crossers: Selection = 2', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR, event_norm=True, density=False)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR, event_norm=True, density=False)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel < 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR, event_norm=True, density=False)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR, event_norm=True, density=False)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Selection < 2', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Stopping Muon Sample: Selection = 0', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 1"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Anode-Cathode Crossers: Selection = 1', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Other Cathode Crossers: Selection = 2', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR, event_norm=True, density=False)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR, event_norm=True, density=False)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Stopping Muon Sample: Selection = 0', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 1"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR, density=False, event_norm=True)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR, density=False, event_norm=True)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Anode-Cathode Crossers: Selection = 1', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR, density=False, event_norm=True)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR, density=False, event_norm=True)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Other Cathode Crossers: Selection = 2', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR, event_norm=True, density=False)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR, event_norm=True, density=False)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(loc="lower center", fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Stopping Muon Sample: Selection = 0', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 1"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Anode-Cathode Crossers: Selection = 1', fontsize=20)
plt.legend(loc='upper center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Other Cathode Crossers: Selection = 2', fontsize=20)
plt.legend(loc='lower center', fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)

S = "sel == 0"
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL+": Sel = 0", color="blue", density=False, event_norm=True)
S = "sel == 1"
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL+": Sel = 1", color="orange", density=False , event_norm=True)
S = "sel == 2"
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL+": Sel = 2", color="red", density=False, event_norm=True)

plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('Data Categories', fontsize=20)
plt.legend(loc='upper right', fontsize=14)
plt.ylim([0, 0.25])
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)

S = "sel == 0"
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL+": Sel = 0", color="blue", density=False, event_norm=True)
S = "sel == 1"
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL+": Sel = 1", color="orange", density=False , event_norm=True)
S = "sel == 2"
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL+": Sel = 2", color="red", density=False, event_norm=True)

plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Counts/Trigger', fontsize=20)
plt.title('MC Categories', fontsize=20)
plt.legend(loc='upper right', fontsize=14)
plt.ylim([0, 0.25])
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)


plt.hist2d(data_df.query(S)['dirx'].values, data_df.query(S)['diry'].values, bins=(B, B), 
           density=True, norm=LogNorm(vmin=10**-3, vmax=10), cmap='viridis')
plt.colorbar(label='Normalized')
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Track Y-Direction', fontsize=20)
plt.title('Data: Selection = 2', fontsize=20)
plt.grid()
plt.show() 

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)


plt.hist2d(mc_df.query(S)['dirx'].values, mc_df.query(S)['diry'].values, bins=(B, B), 
           density=True, norm=LogNorm(vmin=10**-3, vmax=10), cmap='viridis')
plt.colorbar(label='Normalized')
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Track Y-Direction', fontsize=20)
plt.title('MC: Selection = 2', fontsize=20)
plt.grid()
plt.show() 

# Residual Range

In [ ]:
S = "sel >= 0 and rr > 60"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirz", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirz", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0 and rr > 60"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "diry", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "diry", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track Y-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel >= 0 and rr > 60"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
plt.ylim([0, 0.12])
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 1 and rr > 60"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
#plt.ylim([0, 0.12])
plt.title('Selection = 1', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 2 and rr > 60"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
#plt.ylim([0, 0.12])
plt.title('Selection = 2', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

In [ ]:
S = "sel == 0 and rr > 60"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)
plot_1D_error(data_df, "dirx", B, S, label=DATA_LABEL, color=DATA_COLOR)
plot_1D_error(mc_df, "dirx", B, S, label=MC_LABEL, color=MC_COLOR)
plt.xlabel('Track X-Direction', fontsize=20)
plt.ylabel('Area Normalized', fontsize=20)
#plt.ylim([0, 0.12])
plt.title('Selection = 0', fontsize=20)
plt.legend(fontsize=20)
plt.grid()
plt.show()

## Track z-Direction

In [ ]:
S = "sel >= 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)


plt.hist(mc_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='MC', density=True)
plt.hist(data_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='Data', density=True)

plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized Counts', fontsize=20)
plt.title('Full Sample: Selection >= 0', fontsize=20)
plt.legend()
plt.grid()
plt.show()

In [ ]:


S = "sel == 0"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)


plt.hist(mc_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='MC', density=True)
plt.hist(data_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='Data', density=True)

plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized Counts', fontsize=20)
plt.title('Stopping Muon Sample: Selection = 0', fontsize=20)
plt.legend()
plt.grid()
plt.show()

In [ ]:
S = "sel == 1"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)


plt.hist(mc_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='MC', density=True)
plt.hist(data_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='Data', density=True)

plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized Counts', fontsize=20)
plt.title('Anode-Cathode Crossers: Selection = 1', fontsize=20)
plt.legend()
plt.grid()
plt.show()

In [ ]:
S = "sel == 2"
bin_w = 0.1
B = np.arange(-1, 1+bin_w, bin_w)


plt.hist(mc_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='MC', density=True)
plt.hist(data_df.query(S)['dirz'].values, bins=B, histtype='step', linewidth=2, label='Data', density=True)

plt.xlabel('Track Z-Direction', fontsize=20)
plt.ylabel('Area Normalized Counts', fontsize=20)
plt.title('Other Cathode Crossers: Selection = 2', fontsize=20)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.hist2d(data_df['dirz'].values, data_df['rr'].values, bins=(100, 100), range=[[-1, 1], [0, 1000]], norm=LogNorm(), cmap='viridis')
plt.colorbar(label='Counts')
plt.xlabel('dirz')
plt.ylabel('rr')
plt.title('2D Histogram of dirz vs rr for Data')
plt.grid()
plt.show()  

In [ ]:
plt.hist2d(mc_df['dirz'].values, mc_df['rr'].values, bins=(100, 100), range=[[-1, 1], [0, 1000]], norm=LogNorm(), cmap='viridis')
plt.colorbar(label='Counts')
plt.xlabel('dirz')
plt.ylabel('rr')
plt.title('2D Histogram of dirz vs rr for MC')
plt.grid()
plt.show()

In [ ]:
B = np.arange(0, 1000, 10)
plt.hist(mc_df.query('0.6 < abs(dirz) < 0.9')['rr'].values, bins=B, histtype='step', alpha=0.5, label='MC', density=True)
plt.hist(data_df.query('0.6 < abs(dirz) < 0.9')['rr'].values, bins=B, histtype='step', alpha=0.5, label='Data', density=True)
plt.hist(c_df.query('0.6 < abs(dirz) > 0.9')['rr'].values, bins=B, histtype='step', alpha=0.5, label='Corsika', density=True)
plt.xlabel('rr')
plt.ylabel('Density')
#plt.yscale('log')
plt.xlim([0, 100])
plt.title('Comparison of rr Distributions')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.hist(mc_df.query('rr > 60')['dirz'].values, bins=100, range=(-1, 1), histtype='step', alpha=0.5, label='MC dirz', density=True)
plt.hist(data_df.query('rr > 60')['dirz'].values, bins=100, range=(-1, 1), histtype='step', alpha=0.5, label='Data dirz', density=True)
plt.hist(c_df.query('rr > 60')['dirz'].values, bins=100, range=(-1, 1), histtype='step', alpha=0.5, label='Corsika dirz', density=True)
plt.xlabel('dirz')
plt.ylabel('Density')
plt.title('Comparison of dirz Distributions')
plt.legend()
plt.grid()
plt.show()

In [ ]:
def get_Norm(df):

    runs = df["run"].values
    subruns = df["subrun"].values
    evts = df["evt"].values
		
    data = {'run': runs,
		'subrun': subruns,
		'evt': evts}
		
    event_df = pd.DataFrame(data)
    event_df = event_df.drop_duplicates()

    N = event_df.shape[0]
    return N


N_data = get_Norm(data_df)
N_mc = get_Norm(mc_df)
N_c = get_Norm(c_df)

print(f"Number of events in Data: {N_data}")
print(f"Number of events in MC: {N_mc}")
print(f"Number of events in Corsika: {N_c}")

In [ ]:
w = 0.2
B = np.arange(-1, 1+w, w)
tcut = 0

h_data, edges_data = np.histogram(data_df.query('rr > @tcut')['dirz'].values, bins=B)
h_mc, edges_mc = np.histogram(mc_df.query('rr > @tcut')['dirz'].values, bins=B)
h_c, edges_c = np.histogram(c_df.query('rr > @tcut')['dirz'].values, bins=B)

h_data = (h_data / N_data)
h_mc = (h_mc / N_mc)
h_c = (h_c / N_c)


plt.errorbar(edges_data[:-1] + w/2, h_data, yerr=np.sqrt(h_data)/N_data, fmt='o', label='Data')
plt.errorbar(edges_mc[:-1] + w/2, h_mc, yerr=np.sqrt(h_mc)/N_mc, fmt='o', label='MC')
plt.errorbar(edges_c[:-1] + w/2, h_c, yerr=np.sqrt(h_c)/N_c, fmt='o', label='Corsika')
plt.errorbar(edges_mc[:-1] + w/2, h_mc*1.85, yerr=np.sqrt(h_mc)/N_mc, fmt='o', label='MCx1.85')
plt.xlabel('dirz')
plt.ylabel('Counts')
plt.title('Comparison of dirz Distributions')
plt.legend()
plt.grid()
plt.show()
